In [6]:
# Imports y carga
import pandas as pd
import geopandas as gpd
from pathlib import Path
import folium
from folium.plugins import MarkerCluster
import os, sys
import requests
from io import BytesIO
import logging  

In [7]:

cwd = Path.cwd()
repo_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "pyproject.toml").exists():
        repo_root = p
        break

if repo_root:
    os.chdir(repo_root)
    print("Changed working dir to repo root:", repo_root)
else:
    print("Repo root with pyproject.toml not found. Current cwd remains:", cwd)
    print("You can also set repo root manually, e.g.: os.chdir('/home/tepeve/personal/repos/ba_ooh_ads')")
print("Now cwd:", Path.cwd())
print("Python executable:", sys.executable)
print(".venv exists here?:", (Path('.venv').exists()))

Changed working dir to repo root: /home/tepeve/personal/repos/ba_ooh_ads
Now cwd: /home/tepeve/personal/repos/ba_ooh_ads
Python executable: /home/tepeve/personal/repos/ba_ooh_ads/.venv/bin/python
.venv exists here?: True


In [8]:
URL_ETAPAS = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/viajes-etapas-transporte-publico/viajes_BAdata_20241016.csv"


,link_departamento,codigo_provincia
0,2001,2
1,2002,2
2,2003,2
3,2004,2
4,2005,2
...,...,...
62,6778,6
63,6805,6
64,6840,6
65,6861,6


In [9]:
df = pd.read_csv(URL_ETAPAS)

In [10]:
df.columns

Index(['id_tarjeta', 'id_viaje', 'cantidad_etapas', 'rango_horario',
       'etapas_subte', 'etapas_tren', 'etapas_colectivo',
       'longitud_origen_viaje', 'latitud_origen_viaje',
       'longitud_destino_viaje', 'latitud_destino_viaje',
       'departamento_origen_viaje', 'departamento_destino_viaje',
       'factor_expansion_viaje', 'etapas_incompletas', 'genero', 'grupo_edad'],
      dtype='object')

In [11]:
df.departamento_origen_viaje.unique()

array([6028., 6035., 6091., 6098., 6119., 6126., 6134., 6217., 2001.,
       2010., 2011., 2012., 2013., 2014., 2015., 2002., 2003., 2004.,
       2005., 2006., 2007., 2008., 2009., 6245., 6252., 6260., 6266.,
       6270., 6274., 6329., 6364., 6371., 6408., 6410., 6412., 6427.,
       6441., 6434., 6483., 6490., 6497., 6515., 6525., 6532., 6539.,
       6547., 6560., 6568., 6574., 6638., 6648., 6658., 6728., 6735.,
       6749., 6756., 6760., 6778., 6805., 6840., 6861., 6882.,   nan])

In [12]:
df['factor_expansion_viaje'].sum().astype(int)

np.int64(9000918)

In [13]:
conteo_agrupado = df.groupby('rango_horario')['factor_expansion_viaje'].sum().astype(int)
print("Conteo ponderado de registros por rango horario:")
print(conteo_agrupado)

Conteo ponderado de registros por rango horario:
rango_horario
0      29428
1      11209
2       5664
3      14544
4      89956
5     234531
6     489847
7     745762
8     559249
9     440172
10    401855
11    493842
12    721903
13    574433
14    492263
15    528291
16    664148
17    781996
18    606827
19    406463
20    316976
21    214857
22    130115
23     46575
Name: factor_expansion_viaje, dtype: int64


In [14]:
df.grupo_edad.unique()

array([ nan,  75.,  30.,  25.,  20.,  15.,  55.,  65.,  45.,  10.,  80.,
        35.,  40.,  50.,  60.,   5.,  70.,  85.,  90.,   0.,  95., 100.])

In [ ]:
#https://gis.utah.gov/blog/2022-10-26-using-h3-hexes/#:~:text=Creating%20an%20H3%20Hex%20Feature,that%20dataframe%20out%20to%20disk.&text=If%20geopandas%20and%20geojson%20is,his%20process%20into%20Esri%2Dland.

# https://geographicdata.science/book/data/h3_grid/build_sd_h3_grid.html

# https://mathiasschlaeffer.medium.com/dasymetric-spatial-interpolation-in-bigquery-ebd4ca13950d

# https://data.buenosaires.gob.ar/dataset/viajes-etapas-transporte-publico

# https://github.com/alephcero/baCommuteMatrixMap?tab=readme-ov-file

# https://github.com/EL-BID/Matriz-Origen-Destino-Transporte-Publico/tree/main

# https://el-bid.github.io/UrbanTrips/primeros_pasos.html


# https://redatam.indec.gob.ar/

In [15]:
# URL_CENSO='https://www.indec.gob.ar/ftp/cuadros/territorio/codgeo/Codgeo_CABA_con_datos.zip'
URL_CENSO='https://geonode.indec.gob.ar/geoserver/ows?service=WFS&version=2.0.0&request=GetFeature&typename=geonode:radios_censales&outputFormat=shape-zip&srsName=EPSG:4326'
r = requests.get(URL_CENSO)


In [16]:
# https://portalgeoestadistico.indec.gob.ar/maps/geoportal/nota_radios_censales.pdf
gdf = gpd.read_file(BytesIO(r.content))
gdf

/home/tepeve/personal/repos/ba_ooh_ads/.venv/lib/python3.12/site-packages/pyogrio/raw.py:200: RuntimeWarning: /vsizip//vsimem/pyogrio_6b5478cfebe246d2819fe5008408c4bb.zip/radios_censales.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


,fid,id,cpr,jur,cde,dpto,cfn,cro,tro,cod_indec,sag,geometry
0,62484,78625,06,Buenos Aires,06595,Olavarría,21,04,U,065952104,INDEC,"POLYGON ((-60.27547 -36.97623, -60.2742 -36.97..."
1,62485,78232,06,Buenos Aires,06539,Merlo,51,04,U,065395104,INDEC,"POLYGON ((-58.80253 -34.72029, -58.80162 -34.7..."
2,62486,78624,06,Buenos Aires,06595,Olavarría,21,03,U,065952103,INDEC,"POLYGON ((-60.27807 -36.97456, -60.27725 -36.9..."
3,62487,78627,06,Buenos Aires,06595,Olavarría,21,06,R,065952106,INDEC,"POLYGON ((-60.23694 -36.97104, -60.23414 -36.9..."
4,62488,78234,06,Buenos Aires,06539,Merlo,51,06,U,065395106,INDEC,"POLYGON ((-58.79832 -34.73112, -58.79973 -34.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...
66510,62479,77097,06,Buenos Aires,06434,Lanús,09,05,U,064340905,INDEC,"POLYGON ((-58.339 -34.72515, -58.33936 -34.725..."
66511,62480,78623,06,Buenos Aires,06595,Olavarría,21,02,U,065952102,INDEC,"POLYGON ((-60.28281 -36.97828, -60.28287 -36.9..."
66512,62481,78230,06,Buenos Aires,06539,Merlo,51,02,U,065395102,INDEC,"POLYGON ((-58.79486 -34.72708, -58.79519 -34.7..."
66513,62482,78622,06,Buenos Aires,06595,Olavarría,21,01,U,065952101,INDEC,"POLYGON ((-60.29135 -36.98414, -60.29021 -36.9..."


In [19]:
gdf_rcensales = gdf.query("cpr == '02'")     
gdf_rcensales   

NameError: name 'gdf' is not defined

In [9]:
gdf[['cpr','jur']].drop_duplicates()

,cpr,jur
0,06,Buenos Aires
1,50,Mendoza
2,74,San Luis
5,90,Tucumán
6,30,Entre Ríos
9,22,Chaco
15,54,Misiones
23,14,Córdoba
33,02,Ciudad Autónoma de Buenos Aires
60,66,Salta


In [6]:
URL_DICCIONARIO = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/viajes-etapas-transporte-publico/departamentos_exportar.csv'
dptos = pd.read_csv(URL_DICCIONARIO, sep=';')
dptos = dptos[['link_departamento','codigo_provincia']]     


In [7]:
dptos

,link_departamento,codigo_provincia
0,2001,2
1,2002,2
2,2003,2
3,2004,2
4,2005,2
...,...,...
62,6778,6
63,6805,6
64,6840,6
65,6861,6


In [8]:
df.info()

NameError: name 'df' is not defined

In [44]:
from utils.utils_spatial import add_h3_index


df_etapas['origen_h3'] = add_h3_index(df_etapas, lat_col='latitud_origen_viaje', lon_col='longitud_origen_viaje', resolution=H3RESOL)
df_etapas['destino_h3'] = add_h3_index(df_etapas, lat_col='latitud_destino_viaje', lon_col='longitud_destino_viaje', resolution=H3RESOL)


ModuleNotFoundError: No module named 'utils'

In [5]:
df

NameError: name 'df' is not defined

In [ ]:
# https://github.com/nlebovits/ciut-redatam?tab=readme-ov-file
# https://www.indec.gob.ar/indec/web/Institucional-Indec-BasesDeDatos-6

In [15]:
import duckdb
import pandas as pd
import geopandas as gpd

# Configurar DuckDB
con = duckdb.connect()
for cmd in [
    "INSTALL spatial",
    "LOAD spatial", 
    "INSTALL httpfs",
    "LOAD httpfs"
]:
    con.execute(cmd)




In [16]:

query = """
SELECT distinct 
    codigo_variable
FROM 's3://arg-fulbright-data/censo-argentino-2022/censo-2022-largo.parquet'
"""

variables = con.execute(query).fetchdf()
variables   

,codigo_variable
0,PERSONA_MNI
1,VIVIENDA_TOTPOBV
2,VIVIENDA_CODGL
3,DPTO_NDPTO
4,HOGAR_H23
...,...
81,HOGAR_H14
82,PERSONA_P12
83,VIVIENDA_V02
84,PERSONA_P06


In [ ]:

# PERSONA_EDADQUI #Edad en grupos quinquenales
# PERSONA_P02 # Sexo registrado al nacer


In [20]:

query_gender = """
SELECT 
    id_geo,
    etiqueta_categoria,
    SUM(conteo) AS total_conteo
FROM 's3://arg-fulbright-data/censo-argentino-2022/censo-2022-largo.parquet'
WHERE codigo_variable = 'PERSONA_P02'
  AND valor_provincia = '02'
GROUP BY id_geo, etiqueta_categoria
ORDER BY id_geo, etiqueta_categoria;
"""

census_gender = con.execute(query_gender).fetchdf()
  

In [21]:
census_gender

,id_geo,etiqueta_categoria,total_conteo
0,020070101,Mujer / Femenino,114.0
1,020070101,Varón / Masculino,130.0
2,020070201,Mujer / Femenino,313.0
3,020070201,Varón / Masculino,307.0
4,020070202,Mujer / Femenino,265.0
...,...,...,...
7635,021052111,Varón / Masculino,393.0
7636,021052112,Mujer / Femenino,316.0
7637,021052112,Varón / Masculino,295.0
7638,021052113,Mujer / Femenino,397.0


https://censo.gob.ar/index.php/datos_definitivos_caba/

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd

# Configurar DuckDB
con = duckdb.connect()
for cmd in [
    "INSTALL spatial",
    "LOAD spatial", 
    "INSTALL httpfs",
    "LOAD httpfs"
]:
    con.execute(cmd)


query_age = """
SELECT 
    id_geo,
    etiqueta_categoria as tramo_edad,
    SUM(conteo) AS total_conteo
FROM 's3://arg-fulbright-data/censo-argentino-2022/censo-2022-largo.parquet'
WHERE codigo_variable = 'PERSONA_EDADQUI'
  AND valor_provincia = '02'
GROUP BY id_geo, etiqueta_categoria
ORDER BY id_geo, etiqueta_categoria;
"""

census_age = con.execute(query_age).fetchdf()


# Ahora vamos a proyectar la cantidad de hombres y mujeres por tramo etario 
# usando las tasas de feminidad en CABA para el Censo 2022.
# https://censo.gob.ar/index.php/datos_definitivos_caba/
# DataFrame de Tasas de Feminidad (Armado a mano con los datos del archivo)
# https://censo.gob.ar/wp-content/uploads/2023/11/c2022_caba_est_c4_1.xlsx

# Tasas de feminidad por tramo etario en CABA
data_tasas = {
    'tramo_edad': [
        '00 A 04', '05 A 09', '10 A 14', '15 A 19', '20 A 24', '25 A 29', '30 A 34', '35 A 39',
        '40 A 44', '45 A 49', '50 A 54', '55 A 59', '60 A 64', '65 A 69', '70 A 74',
        '75 A 79', '80 A 84', '85 A 89', '90 A 94', '95 A 99', '100 Y MÁS'
    ],
    'tasa_feminidad': [
        97, 97, 97, 101, 108, 109, 108, 106,
        110, 115, 118, 123, 126, 137, 147,
        163, 187, 223, 290, 370, 557
    ]
}

df_tasas = pd.DataFrame(data_tasas)


# Vamos a recategorizar los tramos de edad más altos de la información de cada radio censal
# para que coincidan con los datos de tasas de feminidad.
census_age['tramo_edad'] = census_age['tramo_edad'].replace(
    {'100 A 104': '100 Y MÁS', '105 Y MÁS': '100 Y MÁS'}
)

# Group by id_geo and etiqueta_categoria to unify the counts
census_age = census_age.groupby(['id_geo', 'tramo_edad'], as_index=False)['total_conteo'].sum()


# Ahora hacemos un merge de census_age con df_tasas para luego calcular las proyecciones 
# de hombres y mujeres en cada radio censal y tramo etario.
census_age_by_gender = census_age.merge(df_tasas, left_on='tramo_edad', right_on='tramo_edad', how='left')

# Calculamos la cantidad de hombres y mujeres usando la tasa de feminidad
# Fórmula: Tasa Fem = (Mujeres / Hombres) * 100
# Total = Mujeres + Hombres
# Hombres = Total / (1 + (Tasa Fem / 100))
census_age_by_gender['hombres_float'] = census_age_by_gender['total_conteo'] / (1 + (census_age_by_gender['tasa_feminidad'] / 100))
# Redondeamos hombres al entero más cercano
census_age_by_gender['hombres'] = census_age_by_gender['hombres_float'].round().astype(int)
# Mujeres = Total - Hombres
census_age_by_gender['mujeres'] = census_age_by_gender['total_conteo'] - census_age_by_gender['hombres']

# chequeamos que la suma de hombres y mujeres dé el total original en cada radio censal y grupo etario
check = (census_age_by_gender['hombres'] + census_age_by_gender['mujeres']) == census_age_by_gender['total_conteo']
print(f"Registros con errores de suma: {len(check) - check.sum()}")

# Limpieza de columnas auxiliares
census_age_by_gender = census_age_by_gender.drop(columns=['hombres_float'])

Registros con errores de suma: 0


In [45]:
census_age_by_gender

,id_geo,tramo_edad,total_conteo,tasa_feminidad,hombres,mujeres
0,020070101,00 A 04,11.0,97,6,5.0
1,020070101,05 A 09,21.0,97,11,10.0
2,020070101,10 A 14,29.0,97,15,14.0
3,020070101,15 A 19,15.0,101,7,8.0
4,020070101,20 A 24,16.0,108,8,8.0
...,...,...,...,...,...,...
75561,021052113,75 A 79,26.0,163,10,16.0
75562,021052113,80 A 84,18.0,187,6,12.0
75563,021052113,85 A 89,10.0,223,3,7.0
75564,021052113,90 A 94,5.0,290,1,4.0


In [3]:
PROCESSED_DATA_DIR = Path("data/processed")

pop_reach_h3 = pd.read_parquet(PROCESSED_DATA_DIR / "population_reach_h3.parquet")



In [4]:
pop_reach_h3

,hex_id,tramo_edad,total_conteo_h3_part,hombres_h3_part,mujeres_h3_part,hombres_h3,total_h3,mujeres_h3
0,89c2e302003ffff,00 A 04,1.112517,0.578179,0.534338,1,1,0
1,89c2e302003ffff,05 A 09,1.256148,0.647452,0.608696,1,1,0
2,89c2e302003ffff,10 A 14,1.648789,0.846315,0.802474,1,2,1
3,89c2e302003ffff,15 A 19,1.761902,0.861573,0.900329,1,2,1
4,89c2e302003ffff,20 A 24,1.802591,0.876832,0.925759,1,2,1
...,...,...,...,...,...,...,...,...
56745,89c2e3c49a7ffff,70 A 74,3.194337,1.344984,1.849353,1,3,2
56746,89c2e3c49a7ffff,75 A 79,2.185599,0.840615,1.344984,1,2,1
56747,89c2e3c49a7ffff,80 A 84,0.840615,0.336246,0.504369,0,1,1
56748,89c2e3c49a7ffff,85 A 89,0.840615,0.336246,0.504369,0,1,1
